In [1]:
import pytesseract
from PIL import Image
import re
import base64
from io import BytesIO
from ultralyticsplus import YOLO

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Load the YOLO model for table detection
model = YOLO('foduucom/table-detection-and-extraction')

class FindNutrition:
    def __init__(self, image, text: list):
        self.image = image
        self.text = text

    def detect_tables(self):
        # Perform table detection
        results = model.predict(self.image)
        return results[0].boxes

    def crop_table(self, box):
        # Extract the bounding box coordinates and convert to integers
        left, top, right, bottom = map(int, box.xyxy[0])
        return self.image.crop((left, top, right, bottom))

    def ocr(self, image):
        data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
        result = []
        for i, word in enumerate(data['text']):
            if word.strip():  # Filter out empty results
                result.append({
                    "word": word,
                    "confidence": data['conf'][i],
                    "bounding_box": {
                        "left": data['left'][i],
                        "top": data['top'][i],
                        "width": data['width'][i],
                        "height": data['height'][i]
                    }
                })
        return result

    def plain_text_ocr(self, image):
        return pytesseract.image_to_string(image)

    def find_text_ocr(self, image):
        found_text = []
        data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
        for i, word in enumerate(data['text']):
            if word.strip():  # Filter out empty results
                for j in self.text:
                    if j.lower() in word.lower():
                        found_text.append(word)
        return found_text
    
    def extract_nutritional_content(self, ocr_results):
        keywords = [
            # Macronutrients
            'dietary fiber', 'sugars', 'soluble fiber', 'insoluble fiber',
            'monounsaturated fat', 'polyunsaturated fat', 'trans fat',
            'saturated fat', 'total fat', 'total carbohydrate', 'protein',
            'total calories', 'calories from fat', 'fat',

            # Vitamins
            'vitamin a', 'vitamin c', 'vitamin d', 'vitamin e',
            'vitamin k', 'vitamin b6', 'vitamin b12',
            'thiamin', 'riboflavin', 'niacin', 'folic acid', 'folate',
            'biotin', 'pantothenic acid',

            # Minerals
            'calcium', 'iron', 'magnesium', 'potassium',
            'sodium', 'zinc', 'phosphorus', 'copper', 'manganese',
            'selenium', 'iodine', 'chromium', 'molybdenum',

            # Other Nutrition Components
            'cholesterol', 'added sugars', 'sugar alcohols',
            'omega-3 fatty acids', 'omega-6 fatty acids',
            'other carbohydrate', 'alcohol', 'total sugars', 'dietary fiber',

            # Common Ingredients
            'milk', 'eggs', 'fish', 'shellfish', 'tree nuts',
            'peanuts', 'wheat', 'soy', 'gluten', 'sesame', 'corn',
            'rice', 'yeast', 'flour', 'sugar', 'salt', 'oil',
            'butter', 'cream', 'honey', 'vinegar', 'cheese',
            'yogurt', 'gelatin', 'beef', 'pork', 'chicken', 'lamb',

            # Additives and Flavorings
            'artificial flavors', 'natural flavors', 'preservatives',
            'coloring', 'emulsifiers', 'stabilizers', 'additives',
            'sweeteners', 'lecithin', 'sorbitol', 'maltodextrin',
            'xanthan gum', 'agar', 'pectin', 'caramel color',

            # Spices and Herbs
            'cinnamon', 'basil', 'oregano', 'parsley', 'pepper',
            'turmeric', 'ginger', 'garlic', 'onion', 'coriander',
            'paprika', 'cardamom', 'clove', 'nutmeg', 'bay leaf',

            # Serving Information
            'serving size', 'servings per container', 'daily value',
            'percent daily value', 'calories per serving',

            # Labels (for Identification)
            'non-gmo', 'organic', 'vegan', 'kosher', 'halal', 'gluten-free',
            'low fat', 'low sodium', 'sugar-free', 'no added sugar'

            # Common Allergens
            'milk', 'eggs', 'fish', 'shellfish', 'tree nuts',
            'peanuts', 'wheat', 'soy', 'gluten', 'sesame',

            # Specific Tree Nuts
            'almonds', 'cashews', 'walnuts', 'pecans',
            'hazelnuts', 'pistachios', 'macadamia nuts',

            # Shellfish and Seafood
            'shrimp', 'crab', 'lobster', 'oysters', 'scallops',
            'mussels', 'squid', 'clams', 'anchovies', 'cod',
            'salmon', 'tuna', 'sardines', 'trout', 'haddock',

            # Dairy Products
            'butter', 'cream', 'cheese', 'yogurt', 'whey', 'casein',
            'lactose', 'milk powder', 'skim milk', 'whole milk',

            # Grains (Potential Allergens)
            'wheat', 'barley', 'rye', 'oats', 'corn', 'rice',
            'buckwheat', 'spelt', 'quinoa', 'millet', 'amaranth',

            # Soy Products
            'soybeans', 'soy milk', 'tofu', 'soy sauce', 'edamame',
            'soy protein', 'soy lecithin',

            # Food Additives
            'artificial flavors', 'natural flavors', 'preservatives',
            'coloring', 'stabilizers', 'emulsifiers', 'sweeteners',
            'lecithin', 'sorbitol', 'xanthan gum', 'maltodextrin',
            'carrageenan', 'gelatin', 'pectin', 'aspartame',
            'sucralose', 'acesulfame potassium',

            # Common Oils and Fats
            'vegetable oil', 'palm oil', 'canola oil', 'soybean oil',
            'olive oil', 'coconut oil', 'sunflower oil', 'butter',

            # Sweeteners and Sugars
            'sugar', 'high fructose corn syrup', 'honey', 'agave',
            'maple syrup', 'molasses', 'brown sugar', 'cane sugar',
            'invert sugar', 'glucose', 'fructose', 'lactose',
            'maltose', 'dextrose', 'sucrose',

            # Eggs and Egg Products
            'whole egg', 'egg whites', 'egg yolks', 'powdered egg',

            # Meat and Meat Byproducts
            'beef', 'pork', 'chicken', 'turkey', 'lamb',
            'gelatin', 'broth', 'stock', 'sausage', 'bacon',

            # Spices and Seasonings
            'salt', 'pepper', 'garlic', 'onion', 'cinnamon',
            'nutmeg', 'clove', 'paprika', 'cumin', 'turmeric',
            'oregano', 'basil', 'parsley', 'thyme', 'chili',

            # Legumes (Potential Allergens)
            'lentils', 'chickpeas', 'peas', 'black beans',
            'kidney beans', 'pinto beans', 'lupin',

            # Seeds
            'sesame', 'sunflower seeds', 'pumpkin seeds', 'chia seeds',
            'flaxseeds', 'hemp seeds', 'poppy seeds',

            # Miscellaneous Ingredients
            'yeast', 'vinegar', 'cocoa', 'chocolate', 'coffee',
            'tea', 'mushrooms', 'lemon', 'lime', 'apple',
            'banana', 'strawberry', 'blueberry', 'orange',

            # Gluten-Free Alternatives
            'corn starch', 'potato starch', 'tapioca', 'arrowroot',
            'almond flour', 'coconut flour'
        ]

        nutritional_content = set()
        for word in ocr_results:
            word_lower = word['word'].lower()
            if any(keyword in word_lower for keyword in keywords):
                nutritional_content.add(word['word'])
        return nutritional_content
    
    

c:\Users\phumi\anaconda3\envs\ocr\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\phumi\anaconda3\envs\ocr\Lib\site-packages\ultralytics\nn\tasks.py:340: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals

In [2]:
# Example usage in code:
image = Image.open('./images/n1.jpg')
nutrition_finder = FindNutrition(image, ['calories', 'fat', 'sugar'])
tables = nutrition_finder.detect_tables()
if not tables:
    image_to_use = image
    ocr_results = nutrition_finder.find_text_ocr(image_to_use)
    ocr_results = [l.lower() for l in ocr_results]
    results = set(ocr_results) 
    print(results)
else:
    for table in tables:
        cropped_table = nutrition_finder.crop_table(table)
        ocr_results = nutrition_finder.find_text_ocr(cropped_table)
        ocr_results = [l.lower() for l in ocr_results]
        results = set(ocr_results)
        print(results)

Ultralytics YOLOv8.0.43  Python-3.12.8 torch-2.5.1+cpu CPU
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

0: 640x512 (no detections), 250.4ms
Speed: 2.0ms preprocess, 250.4ms inference, 26.4ms postprocess per image at shape (1, 3, 640, 640)


{'fat', 'sugars', 'calories', 'sugars]'}


In [33]:
import requests
from PIL import Image, UnidentifiedImageError
import base64
from io import BytesIO
import pytesseract

class FindNutrition:
    def __init__(self, image, text: list):
        self.image = image
        self.text = text

    def ocr(self, image):
        data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
        result = []
        for i, word in enumerate(data['text']):
            if word.strip():  # Filter out empty results
                result.append({
                    "word": word,
                    "confidence": data['conf'][i],
                    "bounding_box": {
                        "left": data['left'][i],
                        "top": data['top'][i],
                        "width": data['width'][i],
                        "height": data['height'][i]
                    }
                })
        return result

    def extract_nutritional_content(self, ocr_results):
        keywords = [
            # Macronutrients
            'dietary fiber', 'sugars', 'soluble fiber', 'insoluble fiber',
            'monounsaturated fat', 'polyunsaturated fat', 'trans fat',
            'saturated fat', 'total fat', 'total carbohydrate', 'protein',
            'total calories', 'calories from fat', 'fat',

            # Vitamins
            'vitamin a', 'vitamin c', 'vitamin d', 'vitamin e',
            'vitamin k', 'vitamin b6', 'vitamin b12',
            'thiamin', 'riboflavin', 'niacin', 'folic acid', 'folate',
            'biotin', 'pantothenic acid',

            # Minerals
            'calcium', 'iron', 'magnesium', 'potassium',
            'sodium', 'zinc', 'phosphorus', 'copper', 'manganese',
            'selenium', 'iodine', 'chromium', 'molybdenum',

            # Other Nutrition Components
            'cholesterol', 'added sugars', 'sugar alcohols',
            'omega-3 fatty acids', 'omega-6 fatty acids',
            'other carbohydrate', 'alcohol', 'total sugars', 'dietary fiber',

            # Common Ingredients
            'milk', 'eggs', 'fish', 'shellfish', 'tree nuts',
            'peanuts', 'wheat', 'soy', 'gluten', 'sesame', 'corn',
            'rice', 'yeast', 'flour', 'sugar', 'salt', 'oil',
            'butter', 'cream', 'honey', 'vinegar', 'cheese',
            'yogurt', 'gelatin', 'beef', 'pork', 'chicken', 'lamb',

            # Additives and Flavorings
            'artificial flavors', 'natural flavors', 'preservatives',
            'coloring', 'emulsifiers', 'stabilizers', 'additives',
            'sweeteners', 'lecithin', 'sorbitol', 'maltodextrin',
            'xanthan gum', 'agar', 'pectin', 'caramel color',

            # Spices and Herbs
            'cinnamon', 'basil', 'oregano', 'parsley', 'pepper',
            'turmeric', 'ginger', 'garlic', 'onion', 'coriander',
            'paprika', 'cardamom', 'clove', 'nutmeg', 'bay leaf',

            # Serving Information
            'serving size', 'servings per container', 'daily value',
            'percent daily value', 'calories per serving',

            # Labels (for Identification)
            'non-gmo', 'organic', 'vegan', 'kosher', 'halal', 'gluten-free',
            'low fat', 'low sodium', 'sugar-free', 'no added sugar',

            # Common Allergens
            'milk', 'eggs', 'fish', 'shellfish', 'tree nuts',
            'peanuts', 'wheat', 'soy', 'gluten', 'sesame',

            # Specific Tree Nuts
            'almonds', 'cashews', 'walnuts', 'pecans',
            'hazelnuts', 'pistachios', 'macadamia nuts',

            # Shellfish and Seafood
            'shrimp', 'crab', 'lobster', 'oysters', 'scallops',
            'mussels', 'squid', 'clams', 'anchovies', 'cod',
            'salmon', 'tuna', 'sardines', 'trout', 'haddock',

            # Dairy Products
            'butter', 'cream', 'cheese', 'yogurt', 'whey', 'casein',
            'lactose', 'milk powder', 'skim milk', 'whole milk',

            # Grains (Potential Allergens)
            'wheat', 'barley', 'rye', 'oats', 'corn', 'rice',
            'buckwheat', 'spelt', 'quinoa', 'millet', 'amaranth',

            # Soy Products
            'soybeans', 'soy milk', 'tofu', 'soy sauce', 'edamame',
            'soy protein', 'soy lecithin',

            # Food Additives
            'artificial flavors', 'natural flavors', 'preservatives',
            'coloring', 'stabilizers', 'emulsifiers', 'sweeteners',
            'lecithin', 'sorbitol', 'xanthan gum', 'maltodextrin',
            'carrageenan', 'gelatin', 'pectin', 'aspartame',
            'sucralose', 'acesulfame potassium',

            # Common Oils and Fats
            'vegetable oil', 'palm oil', 'canola oil', 'soybean oil',
            'olive oil', 'coconut oil', 'sunflower oil', 'butter',

            # Sweeteners and Sugars
            'sugar', 'high fructose corn syrup', 'honey', 'agave',
            'maple syrup', 'molasses', 'brown sugar', 'cane sugar',
            'invert sugar', 'glucose', 'fructose', 'lactose',
            'maltose', 'dextrose', 'sucrose',

            # Eggs and Egg Products
            'whole egg', 'egg whites', 'egg yolks', 'powdered egg',

            # Meat and Meat Byproducts
            'beef', 'pork', 'chicken', 'turkey', 'lamb',
            'gelatin', 'broth', 'stock', 'sausage', 'bacon',

            # Spices and Seasonings
            'salt', 'pepper', 'garlic', 'onion', 'cinnamon',
            'nutmeg', 'clove', 'paprika', 'cumin', 'turmeric',
            'oregano', 'basil', 'parsley', 'thyme', 'chili',

            # Legumes (Potential Allergens)
            'lentils', 'chickpeas', 'peas', 'black beans',
            'kidney beans', 'pinto beans', 'lupin',

            # Seeds
            'sesame', 'sunflower seeds', 'pumpkin seeds', 'chia seeds',
            'flaxseeds', 'hemp seeds', 'poppy seeds',

            # Miscellaneous Ingredients
            'yeast', 'vinegar', 'cocoa', 'chocolate', 'coffee',
            'tea', 'mushrooms', 'lemon', 'lime', 'apple',
            'banana', 'strawberry', 'blueberry', 'orange',

            # Gluten-Free Alternatives
            'corn starch', 'potato starch', 'tapioca', 'arrowroot',
            'almond flour', 'coconut flour'
        ]

        nutritional_content = set()
        for word in ocr_results:
            word_lower = word['word'].lower()
            if any(keyword in word_lower for keyword in keywords):
                nutritional_content.add(word['word'])
        return nutritional_content

def final_info(image_data_base64, user_id, allergy: list):
    # Decode base64 image
    try:
        image = Image.open(BytesIO(base64.b64decode(image_data_base64)))
    except (base64.binascii.Error, UnidentifiedImageError) as e:
        return {"error": "Invalid image data"}

    # Initialize the OCR class
    nutrition_finder = FindNutrition(image, [])

    # Get plain text OCR results
    ocr_results = nutrition_finder.ocr(image)

    # Extract nutritional content
    nutritional_content = nutrition_finder.extract_nutritional_content(ocr_results)


    # Prepare the payload for the allergy API
    payload = {
        "ingredients": ["Egg", "Coconut milk"]  # Replace with actual ingredients if needed
    }

    # Make a request to the other API to get allergy information
    allergy_api_url = "https://recommend-api-production.up.railway.app/allergy/ingredient"
    headers = {'Content-Type': 'application/json'}
    response = requests.post(allergy_api_url, json=payload, headers=headers)
    if response.status_code != 200:
        return {"error": "Failed to fetch allergy information"}

    allergy_info = response.json()

    # Make a request to the other API to get option
    pop_option = "https://recommend-api-production.up.railway.app/recommend/popularity"  # Replace with the actual URL of the other API
    response = requests.get(pop_option)
    if response.status_code != 200:
        return {"error": "Failed to fetch pop option information"}

    pop_option_info = response.json()

    # Check if any of the ingredients are allergens
    # Check from the allergy API response and allergy that user input
    # Check for common allergens
    warning = False
    allergens = {allergen.lower() for allergen in allergy_info.get("allergy", [])}
    user_allergens = list(allergy)
    user_allergens = {allergen.lower() for allergen in user_allergens}
    common_allergens = allergens.intersection(user_allergens)
    if common_allergens:
        warning = True

    # Combine the results
    final_result = {
        "user_id": user_id,
        "allergy": allergy,
        "warning": warning,
        "allergy_info": allergy_info.get("allergy", []),
        "best_restaurant": pop_option_info.get("best_restaurant", []),
        "nutritional_content": list(nutritional_content)
    }
    return final_result

In [35]:
# Example usage
image_data_base64 = "..."  # Replace with actual base64 string
result = final_info(image_data_base64, 'Davin', ['egg'])
print(result)

allergen {'coconut', 'egg'}
user {'egg'}
common_allergen {'egg'}
{'user_id': 'Davin', 'allergy': ['egg'], 'warning': True, 'allergy_info': ['egg', 'coconut'], 'best_restaurant': {'menu': [{'dish': 'Chili Dip with Fried Mackerel', 'price': '227.50 THB'}, {'dish': 'Massaman Curry', 'price': '332.50 THB'}, {'dish': 'Stir-Fried Rice Noodles', 'price': '245.00 THB'}, {'dish': 'Grilled Chicken Skewers', 'price': '262.50 THB'}, {'dish': 'Tofu Pudding in Ginger Syrup', 'price': '192.50 THB'}], 'name': 'Thai Royal Kitchen'}, 'nutritional_content': ['Sugars', 'Potassium', 'Protein', 'Calcium', 'Fat', 'Sugars]', 'Iron', 'Sodium', 'Cholesterol']}
